In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, glob, os, scipy.stats as stats
from neo.rawio import BlackrockRawIO
from neo.io import BlackrockIO


### setup

In [7]:
# 1. Define your trigger‐to‐name mapping
trigger_map = {
    10: 'Block started',
    20: 'Baseline started',
    30: 'Stim started',
    40: 'Delay started',
    50: 'Task started',
    51: 'Marker moved',
    52: 'Left pressed',
    53: 'Left released',
    54: 'Right pressed',
    55: 'Right released',
    56: 'Response submitted',
    60: 'Anticipation started',
    70: 'Feedback started',
    80: 'Block ended'
}

# 2. Load the session with BlackrockIO (no extension on filename)
io = BlackrockIO(filename='../results/202509/20250523-141833-002')
# io = BlackrockIO(filename='../results/20250717/20250717-143139-001')
block = io.read_block(lazy=False)
seg = block.segments[0]  # Assuming only one segment


### timestamps

In [8]:
# Find the digital‐input event stream
dig_ev = None
for ev in seg.events:
    if 'digital' in ev.name.lower():
        dig_ev = ev
        break
if dig_ev is None:
    raise RuntimeError("No digital‐input events found in NEV")

# Extract times (sec) and integer codes
times = dig_ev.times
codes = dig_ev.labels.astype(int)

# Sanity‐check: all expected codes are present
expected_codes = set(trigger_map.keys())
found_codes    = set(codes.tolist())
missing = expected_codes - found_codes
# if missing:
#     raise RuntimeError(f"Missing trigger codes: {sorted(missing)}")

# Build your trigger‐times dictionary
trig_times = {
    name: times[codes == code]
    for code, name in trigger_map.items()
}

# 7. (Optional) Example: print all Block‐start times
print("Blocks starts:", trig_times['Block started'])

# 8. Print block-starts in mins
print("Block starts (mins):", trig_times['Block started'] / 60)

Blocks starts: [  37.26826667  299.53076667  558.33186667  809.76626667 1093.16743333
 1339.7519    ] s
Block starts (mins): [ 0.62113778  4.99217944  9.30553111 13.49610444 18.21945722 22.32919833] s


### spikes

In [11]:
# Check what you have
print(f"Number of spiketrains: {len(seg.spiketrains)}")
for i, st in enumerate(seg.spiketrains):
    print(f"Unit {i}: {st.name}, {len(st)} spikes")
    
# Check the naming pattern
if len(seg.spiketrains) > 0:
    first_name = seg.spiketrains[0].name
    print(f"First spiketrain name: '{first_name}'")
    
    # Check for waveforms (if spike sorting was done)
    if hasattr(seg.spiketrains[0], 'waveforms') and seg.spiketrains[0].waveforms is not None:
        print("Has waveforms - likely sorted units")
    else:
        print("No waveforms - likely threshold crossings")

Number of spiketrains: 0


### lfp

In [ ]:
for sig in seg.analogsignals:
    print(sig.name, sig.sampling_rate, sig.shape)

# e.g. to grab channel 0’s full voltage trace:
lfp0 = seg.analogsignals[0].magnitude
t0   = seg.analogsignals[0].times

# 5. Epoch per phase/trial:
#    Suppose you want LFP between “Task started” and “Anticipation started” for trial i:
trial_onset = trig_times['Task started'][i]
trial_offset= trig_times['Anticipation started'][i]
mask = (t0 >= trial_onset) & (t0 < trial_offset)
epoch0 = lfp0[mask]  # this trial’s LFP for channel 0

# Done—you’ve now got both your event marks and NSx‐derived continuous signals
# all from the same Block. You can repeat for spikes (seg.spiketrains) or other channels.

Channels: (chan1 chan2 chan3 chan4 chan5 chan6 chan7 chan8 chan9 chan10 chan11 chan12 chan13 chan14 chan15 chan16 chan17 chan18 chan19 chan20 chan21 chan22 chan23 chan24 chan25 chan26 chan27 chan28 chan29 chan30 chan31 chan32 chan33 chan34 chan35 chan36 chan37 chan38 chan39 chan40 chan41 chan42 chan43 chan44 chan45 chan46 chan47 chan48 chan49 chan50 chan51 chan52 chan53 chan54 chan55 chan56 chan57 chan58 chan59 chan60 chan61 chan62 chan63 chan64 chan65 chan66 chan67 chan68 chan69 chan70 chan71 chan72 chan73 chan74 chan75 chan76 chan77 chan78 chan79 chan80 chan81 chan82 chan83 chan84 chan85 chan86 chan87 chan88 chan89 chan90 chan91 chan92 chan93 chan94 chan95 chan96 chan97 chan98 chan99 chan100 chan101 chan102 chan103 chan104 chan105 chan106 chan107 chan108 chan109 chan110 chan111 chan112 chan113 chan114 chan115 chan116 chan117 chan118 chan119 chan120 chan121 chan122 chan123 chan124 chan125 chan126 chan127 chan128) 1000.0 Hz (1555591, 128)
Channels: (EyeX EyeY Pupil BP) 1000.0 Hz (15555